In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import urllib
import ast
import numpy as np
from tqdm import tqdm
import datetime


code_data = pd.read_csv("code_data.csv")

#numdays = 3 #can be any number of days up to when Baidu has data
#base = datetime.datetime.today()
#dates_sep = [str(base - datetime.timedelta(days=x)).split(" ")[0] for x in range(numdays)]
#dates = [str(base - datetime.timedelta(days=x)).split(" ")[0].replace("-","") for x in range(numdays)]

dates_sep = ['2023-01-20', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27']
dates = [date.replace("-","") for date in dates_sep]

code_data

,City_CH,Prov_CH,Prov_EN,City_Code,City_EN,Prov_Code
0,北京市,北京市,Beijing,110000,Beijing,110000.0
1,天津市,天津市,Tianjin,120000,Tianjin,120000.0
2,石家庄市,河北省,Hebei,130100,Shijiazhuang,130000.0
3,唐山市,河北省,Hebei,130200,Tangshan,130000.0
4,秦皇岛市,河北省,Hebei,130300,Qinhuangdao,130000.0
...,...,...,...,...,...,...
364,可克达拉市,新疆,Xinjiang,659008,Kelada,650000.0
365,昆玉市,新疆,Xinjiang,659009,Kunyu,650000.0
366,台湾省,台湾省,Taiwan,710000,Taiwan,710000.0
367,香港特别行政区,特别行政区,Hongkong,810000,Hongkong,810000.0


In [2]:
###move in data
error_code = []
error_date = []
moveins = []

for d in range(len(dates)):  
    date = dates[d]
    by_cities = []
    for i in tqdm(range(len(code_data))):
        code = code_data.City_Code[i]
        url = "http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id="+str(code)+"&type=move_in&date="+date
        try:
            file = urllib.request.urlopen(url, timeout=20) #increase timeout to avoid connection error
            file = file.read()
            dict_str = file.decode("UTF-8")
            dict_str = dict_str.replace('\ncb({"errno":0,"errmsg":"SUCCESS","data":{"list":[{', "{")
            dict_str = dict_str.replace("]}})", "")
            data = ast.literal_eval(dict_str)
            data = pd.DataFrame(list(data))
            data.columns = ["City_CH", "Prov_CH", "proportion"]
            data["City_EN_origin"] = [code_data[code_data.City_CH==c].City_EN.values[0] for c in data.City_CH]
            data["Prov_EN_origin"] = [code_data[code_data.City_CH==c].Prov_EN.values[0] for c in data.City_CH]
            data["City_EN_destination"] = np.repeat(code_data[code_data.City_Code==code].City_EN.values[0], len(data))
            data["Prov_EN_destination"] = np.repeat(code_data[code_data.City_Code==code].Prov_EN.values[0], len(data))
            data["City_CH_origin"] = [code_data[code_data.City_CH==c].City_CH.values[0] for c in data.City_CH]
            data["Prov_CH_origin"] = [code_data[code_data.City_CH==c].Prov_CH.values[0] for c in data.City_CH]
            data["City_CH_destination"] = data.City_CH.values
            data["Prov_CH_destination"] = data.Prov_CH.values
            data["City_code_origin"] = [code_data[code_data.City_CH==c].City_Code.values[0] for c in data.City_CH]
            data["Prov_code_origin"] = [code_data[code_data.City_CH==c].Prov_Code.values[0] for c in data.City_CH]
            data["City_code_destination"] = np.repeat(code_data[code_data.City_Code==code].City_Code.values[0], len(data))
            data["Prov_code_destination"] = np.repeat(code_data[code_data.City_Code==code].Prov_Code.values[0], len(data))
            by_cities.append(data)
        except:
            error_code.append(code)
            error_date.append(dates_sep[d])
            continue
    errors = pd.DataFrame({"error_code":error_code, "error_date":error_date})
    data = pd.concat(by_cities)
    data['date'] = dates_sep[d]
    moveins.append(data)
data = pd.concat(moveins)
data.to_csv("china_data_movein_2023-01-20_2023-01-27.csv", index=False, encoding='utf_8_sig')


100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [08:32<00:00,  1.39s/it]


In [2]:
###move out data
error_code = []
error_date = []
moveouts = []
for d in range(len(dates)):  
    date = dates[d]
    by_cities = []
    for i in tqdm(range(len(code_data))):
        code = code_data.City_Code[i]
        url = "http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id="+str(code)+"&type=move_out&date="+date
        try:
            file = urllib.request.urlopen(url, timeout=20) #increase timeout to avoid connection error
            file = file.read()
            dict_str = file.decode("UTF-8")
            dict_str = dict_str.replace('\ncb({"errno":0,"errmsg":"SUCCESS","data":{"list":[{', "{")
            dict_str = dict_str.replace("]}})", "")
            data = ast.literal_eval(dict_str)
            data = pd.DataFrame(list(data))
            data.columns = ["City_CH", "Prov_CH", "proportion"]
            data["City_EN_destination"] = [code_data[code_data.City_CH==c].City_EN.values[0] for c in data.City_CH]
            data["Prov_EN_destination"] = [code_data[code_data.City_CH==c].Prov_EN.values[0] for c in data.City_CH]
            data["City_EN_origin"] = np.repeat(code_data[code_data.City_Code==code].City_EN.values[0], len(data))
            data["Prov_EN_origin"] = np.repeat(code_data[code_data.City_Code==code].Prov_EN.values[0], len(data))
            data["City_CH_destination"] = [code_data[code_data.City_CH==c].City_CH.values[0] for c in data.City_CH]
            data["Prov_CH_destination"] = [code_data[code_data.City_CH==c].Prov_CH.values[0] for c in data.City_CH]
            data["City_CH_origin"] = data.City_CH.values
            data["Prov_CH_origin"] = data.Prov_CH.values
            data["City_code_origin"] = [code_data[code_data.City_CH==c].City_Code.values[0] for c in data.City_CH]
            data["Prov_code_origin"] = [code_data[code_data.City_CH==c].Prov_Code.values[0] for c in data.City_CH]
            data["City_code_destination"] = np.repeat(code_data[code_data.City_Code==code].City_Code.values[0], len(data))
            data["Prov_code_destination"] = np.repeat(code_data[code_data.City_Code==code].Prov_Code.values[0], len(data))
            by_cities.append(data)
        except:
            error_code.append(code)
            error_date.append(dates_sep[d])
            continue
    errors = pd.DataFrame({"error_code":error_code, "error_date":error_date})
    data = pd.concat(by_cities)
    data['date'] = dates_sep[d]
    moveouts.append(data)
data = pd.concat(moveouts)
data.to_csv("china_data_moveout_2023-01-20_2023-01-27.csv", index=False, encoding='utf_8_sig')

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [07:00<00:00,  1.14s/it]
